# Inventories in Ansible
---

An Ansible inventory file is a collection of target machines, or hosts, on which Ansible can run. Ansible connects to these targets via SSH (Linux) or WinRM (Windows) in order to run on them.
- Ansible inventory files are written in INI format
- You will need either a domain name or an IP address for each target machine. This includes a username and password where applicable.
- You are able to define groups of targets and groups of groups. A single host can also be a part of multiple groups.
- You are able to define group and host variables. Group variables apply to each host in the group.

In an Ansible playbook, you can run on a single host or a group of hosts. When you run on a group, the subsequent tasks will run on all individual hosts in that group. 

### Example Inventory File
---

``` ini
# this is a stand-alone host
mail.example.com

[webservers]          # this is a group of hosts
server1.example.com
server2.example.com

[dbservers]           # below is a hostvar
one.example.com       fooVar="testABC"
two.example.com
three.example.com

[allservers:children] # this is a group of groups
webservers
dbservers

[allservers:vars]     # this is a groupvar
newVar="test123"
```

## Demos
---

### Adding and Switching Hosts

In this example we will take a look at a common way we can work with Ansible Inventory inside an Ansible Playbook. We will add a host (and hostvar) to an existing inventory file, and proceed to switch to that host.

**1. Run the playbook below**

In [ ]:
wpb "ansible_add_host.yml"

########################  PLAYBOOK  ########################
---
- hosts: localhost
  connection: local

  tasks:
  - name: Add a host with a hostvar
    add_host:
      hostname: newhost
      test_var: "testval"
      ansible_host: 127.0.0.1

- hosts: [newhost]
  connection: local

  tasks:
  - name: Debug the hostvar
    debug:
      var: hostvars['newhost'].test_var
...
############################################################

ansible-playbook "ansible_add_host.yml"

### Host and Groupvars

In this example we will take a look at how to define host and group vars and how they interact with one another.

**1. Run the playbook below**

> Note:
> - when given the same name, hostvars will override groupvars
> - feel free to edit and experiment with the inventory file and playbook as you please!

In [ ]:
cat << EOF > "/home/cvx_admin_user/ansible_hostfile.txt"

########################  HOSTFILE  ########################

[example_group]
localhost   ansible_connection=local my_var="hostvar value"
192.168.42.123

[example_group:vars]
my_var="groupvar value"

############################################################

EOF

In [ ]:

wpb "ansible_variable_test.yml"

########################  PLAYBOOK  ########################
---
- hosts: localhost
  connection: local

  tasks:
  - debug:
      msg: "{{ hostvars['localhost'].my_var }}"

  - debug:
      msg: "{{ hostvars['192.168.42.123'].my_var }}"

- hosts: [example_group]
  connection: local

  tasks:
  - debug:
      msg: "{{ group_names }}"
    run_once: true
...
############################################################

                      # Hosts File             # Playbook to be run
ansible-playbook -i   "/home/cvx_admin_user/ansible_hostfile.txt"   "ansible_variable_test.yml"

# ansible-inventory --list -i "/home/cvx_admin_user/ansible_hostfile.txt"

## Further Explanation
---

**Notes:**
> - You can define groupvars over all hosts using `[all:vars]`
> - Saying 'hostfile' is equivalent to saying 'inventory file'
> - The default Ansible hostfile, if you don't define your own, is located under `/etc/ansible/hosts`
> - You are able to control the order that the machines are run in if you are running on a group of hosts
> - When you switch hosts in an Ansible playbook, by definition you are changing Plays

**See Also:**
- [Intro to Inventories](https://docs.ansible.com/ansible/latest/user_guide/intro_inventory.html)
- [Building an Inventory](https://docs.ansible.com/ansible/latest/network/getting_started/first_inventory.html)
- [How host and group variables are merged](https://docs.ansible.com/ansible/latest/user_guide/intro_inventory.html#how-variables-are-merged)